In [2]:
import pandas as pd

df=pd.read_excel(r'C:\Users\gbo10\Videos\research\counting_research_algorithms\fifty_one\measurements\updated_filtered_data_with_lengths.xlsx')

print(len(df))
df=df.dropna()
print(len(df))

df['Pond_Type'] = df['Pond_Type'].replace({'test-car': 'square-femal', 'test-right': 'circular big male', 'test-left': 'circular small male'})

df.groupby('Pond_Type')['Label'].nunique()

192
148


Pond_Type
circular big male      36
circular small male     5
square-femal           28
Name: Label, dtype: int64

In [6]:
df

,Unnamed: 0,Label,PrawnID,Avg_Length,Std_Length,Uncertainty,Length_1,Length_2,Length_3,BoundingBox_1,...,diff_max_min,diff_mean,diff_max,diff_min,the_min_diff,min difference,min_mpe,max_mpe,min_mae,max_mae
0,8,carapace:undistorted_GX010067_33_625.jpg_gamma,Prawn_0,41.918000,2.531547,1.461589,40.475,39.802,45.477,"(2996.016573, 737.0050179, 159.9996606, 166.00...",...,5.675,3.119784,8.121784,2.446784,2.446784,12.119641,6.147389,17.859102,2.446784,8.121784
1,9,carapace:undistorted_GX010067_33_625.jpg_gamma,Prawn_1,26.698000,1.842735,1.063904,28.855,24.353,26.886,"(3770.0181864, 1859.0067423, 114.9979869, 117....",...,4.502,0.085277,2.054277,2.447723,0.085277,2.103141,0.317179,10.051014,0.085277,2.447723
2,10,carapace:undistorted_GX010067_33_625.jpg_gamma,Prawn_2,46.299667,4.579569,2.644015,51.637,40.454,46.808,"(1136.0077806, 2072.0074935, 280.9986255, 84.9...",...,11.183,3.944933,8.773933,2.409067,2.409067,6.058226,5.955076,16.991563,2.409067,8.773933
3,11,carapace:undistorted_GX010067_33_625.jpg_gamma,Prawn_3,24.278667,0.552181,0.318802,24.729,24.606,23.501,"(368.0014839, 2140.0110291, 141.0003378, 1.002...",...,1.228,2.211272,2.334272,1.106272,1.106272,10.571630,4.707341,9.439412,1.106272,2.334272
4,20,carapace:undistorted_GX010068_26_666.jpg_gamma,Prawn_0,33.843333,2.607538,1.505463,36.364,30.252,34.914,"(1496.0102131, 2248.0127922, 149.0029833, 149....",...,6.112,5.436597,6.886597,0.774597,0.774597,0.306276,2.560481,18.937951,0.774597,6.886597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1077,carapace:undistorted_GX010183_37_685.jpg_gamma,Prawn_2,39.201000,2.805890,1.619981,38.939,35.903,42.761,"(1039.9968915, 2155.9928691, 181.0022403, 212....",...,6.858,18.460292,22.282292,15.424292,15.424292,25.161231,42.961013,52.108913,15.424292,22.282292
188,1086,carapace:undistorted_GX010183_80_1633.jpg_gamma,Prawn_0,39.565333,2.045577,1.181015,41.730,36.821,40.145,"(364.0008550000001, 1448.0027236, 60.9995292, ...",...,4.909,9.841948,11.426948,6.517948,6.517948,32.124619,17.701712,27.383052,6.517948,11.426948
189,1087,carapace:undistorted_GX010183_80_1633.jpg_gamma,Prawn_1,35.726667,2.216436,1.279660,34.318,34.006,38.856,"(2732.0065866, 1332.0003336, 249.000479, 93.00...",...,4.850,4.014948,8.552948,3.702948,3.702948,3.725130,10.889101,22.011909,3.702948,8.552948
190,1096,carapace:undistorted_GX010183_128_2852.jpg_gamma,Prawn_0,32.746333,1.552567,0.896375,34.761,30.983,32.495,"(3943.98895, 1279.9998518, 236.9997988, 124.99...",...,3.778,9.217687,11.483687,7.705687,7.705687,9.271802,24.870695,33.036125,7.705687,11.483687


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, shapiro, levene
# import pingouin as pg
import statsmodels.api as sm
import numpy as np
import plotly.express as px

# Load your data

# Calculate expert averages and standard deviations
df['expert_mean'] = df[['Length_1', 'Length_2', 'Length_3']].median(axis=1)
df['expert_std'] = df[['Length_1', 'Length_2', 'Length_3']].std(axis=1)

df['expert_max'] = df[['Length_1', 'Length_2', 'Length_3']].max(axis=1)
df['expert_min'] = df[['Length_1', 'Length_2', 'Length_3']].min(axis=1)

#diffrence in pixels between the min and max of Length_1_pixels, Length_2_pixels, Length_3_pixels

df['diff_max_min'] =abs(df[['Length_1', 'Length_2', 'Length_3']].max(axis=1) - df[['Length_1', 'Length_2', 'Length_3']].min(axis=1))

print(f'mean difference between min max' ,df['diff_max_min'].mean())
print(f'std  difference between min max ',df['diff_max_min'].std())


#violoin plot of the difference between the min and max of the three measurements
fig = px.violin(df,
                x='Pond_Type', 
                y='diff_max_min', 
                box=True,  # Include box plot
                points="all",  # Show all points
                hover_data=['Label', 'PrawnID'],  # Include hover data
                title=f'Violin plot of diff_max_min',)
#Save the plot to png
fig.write_image(f'violin carapace diff_max_min.png')
fig.show()

#how many Length_ground_truth_annotation(mm) in comparison with lengthe_1, length_2, length_3 bigger than 50%

# df['difference_mean_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_mean']/df['Length_ground_truth_annotation(mm)']) 
# df['difference_max_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_max']/df['Length_ground_truth_annotation(mm)'])
# df['difference_min_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_min']/df['Length_ground_truth_annotation(mm)'])

# df['min difference'] = df[['difference_mean_annotation_expert', 'difference_max_annotation_expert', 'difference_min_annotation_expert']].min(axis=1)
# df['max_difference'] = df[['difference_mean_annotation_expert', 'difference_max_annotation_expert', 'difference_min_annotation_expert']].max(axis=1)    


# df['difference_max_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_max']/df['Length_ground_truth_annotation(mm)'])

# df['difference_min_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_min']/df['Length_ground_truth_annotation(mm)'])
#violin difference_annotation_expert
# fig = px.violin(df,
#                 x='Pond_Type',
#                 y='min difference',
#                 box=True,  # Include box plot
#                 points="all",  # Show all points
#                 hover_data=['Label', 'PrawnID'],  # Include hover data
#                 title=f'Violin plot of difference_annotation_expert',)
# #Save the plot to png
# fig.write_image(f'violin carapace difference_annotation_expert.png')
# fig.show()




# Calculate differences
df['diff_mean'] =abs(df['Length_fov(mm)'] - df['expert_mean'])
df['diff_max'] =abs(df['Length_fov(mm)'] - df['expert_max'])
df['diff_min'] =abs(df['Length_fov(mm)'] - df['expert_min'])

df['the_min_diff'] = df[['diff_mean', 'diff_max', 'diff_min']].min(axis=1)


#the length measurement of the the_min_diff
# df['the_min_diff_length'] =df['Length_fov(mm)'] - df['the_min_diff']

#the length measurement of the the_max_diff
#percent error of the the_min_diff


# df['org_min_diff'] = [
#     df['Length_fov(mm)'].iloc[i] - df['the_min_diff'].iloc[i] 
#     if df['the_min_diff'].iloc[i] > 0 
#     else df['Length_fov(mm)'].iloc[i] + abs(df['the_min_diff'].iloc[i]) 
#     for i in range(len(df))
# ]


df['min difference']=df[['Error_percentage_distance_mm_ground_min', 'Error_percentage_distance_mm_ground_max','Error_percentage_distance_mm_ground_median']].min(axis=1)

# Calculate the percentage error
# df['the_min_diff_percentage_error'] = (df['the_min_diff'].abs() / df['org_min_diff'].abs()) * 100

# # Cap the error at 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error'].apply(lambda x: min(x, 100))

# the percentage error of the the_min_diff when smaller than 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error']
df['min_mpe']=df[['MPError_fov_min', 'MPError_fov_max','MPError_fov_median']].min(axis=1)

df['max_mpe']=df[['MPError_fov_min', 'MPError_fov_max','MPError_fov_median']].max(axis=1)

df['min_mae']=df[['AbsError_fov_min', 'AbsError_fov_max','AbsError_fov_median']].min(axis=1)

df['max_mae']=df[['AbsError_fov_min', 'AbsError_fov_max','AbsError_fov_median']].max(axis=1)


# Create a violin plot using plotly
import plotly.express as px
#violin plots wiht plotly for mean percentage error and MAE
for error in ['min_mpe', 'min_mae', 'max_mpe', 'max_mae']:
        # Create a violin plot using plotly 
    fig = px.violin(df,
                    x='Pond_Type', 
                    y=error, 
                    box=True,  # Include box plot
                    points="all",  # Show all points
                    hover_data=['Label', 'PrawnID'],  # Include hover data
                    title=f'Violin plot of {error}',)
    #Save the plot to png
    fig.write_image(f'violin carapace {error}_.png')
    fig.show()

#change name of each pond type if test-car change to square-female if test-right change to circular big male if test-left change to circular small mal




#scatter plot the_min_diff_percentage_error and  difference_annotation_expert plotly
fig = px.scatter(df, x='min_mpe', y='min difference', color='Pond_Type', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='min_mpe vs. difference_annotation_expert')

#save the plot to png
fig.write_image('the_min_diff_percentage_error vs. difference_annotation_expert.png')
# Add a trendline


fig.show()


#scatter plot the_min_diff_percentage_error and  pose_eval_iou plotly
fig = px.scatter(df, x='min_mpe', y='pose_eval_iou', color='Pond_Type', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='min_mpe vs. pose_eval_iou')

#save the plot to png
fig.write_image('the_min_diff_percentage_error vs. pose_eval_iou.png')
# Add a trendline
fig.show()

#scatter plot between min_mpe and height
fig = px.scatter(df, x='Height(mm)', y='min_mpe', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='min_mpe vs. Height')

#save the plot to png
fig.write_image('min_mpe vs. Height.png')
# Add a trendline
fig.show()





# Normality Test


In [13]:
import plotly.express as px





df['min_mpe_length-'] = df.apply(lambda row: row['Length_fov(mm)'] - row['Length_fov(mm)'] * row['min_mpe']/100, axis=1)
                                
df['min_mpe_length+']= df.apply(lambda row: row['Length_fov(mm)'] + row['Length_fov(mm)'] * row['min_mpe']/100, axis=1)

df['min_mae_length-'] = df.apply(lambda row: row['Length_fov(mm)'] -  row['min_mae'], axis=1)
df['min_mae_length+'] = df.apply(lambda row: row['Length_fov(mm)'] +  row['min_mae'], axis=1)




def get_closest_length_and_distance(row, target_length1,true_length2):
    lengths = [row['Length_1'], row['Length_2'], row['Length_3']]
    closest_length_1 = min(lengths, key=lambda x: abs(x - target_length1))
    distance_1 = abs(closest_length_1 - target_length1)
    closest_length_2 = min(lengths, key=lambda x: abs(x - true_length2))
    distance_2 = abs(closest_length_2 - true_length2)

    if distance_1 < distance_2:
        closest_length = closest_length_1
        distance = distance_1
    else:
        closest_length = closest_length_2
        distance = distance_2


    return closest_length, distance

# Apply the function to find the closest length and distance to min_mpe_length-
df['closest_length_min_mpe'] = df.apply(
    lambda row: (get_closest_length_and_distance(row, row['min_mpe_length-'],row['min_mpe_length+'])), axis=1
)

df['closest_length_min_mae'] = df.apply( lambda row: (get_closest_length_and_distance(row, row['min_mae_length-'],row['min_mae_length+'])), axis=1)

# Apply the function to find the closest length and distance to min_mpe_length+
df['closest_length_min_mae_value']=df['closest_length_min_mae'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
# Assuming df cont



import plotly.graph_objects as go


#scatter plot between length_fov and closest_length_min_mae_value
fig = px.scatter(df,color='Pond_Type',x= 'focal_RealLength(cm)', y='closest_length_min_mae_value', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title=f'pinhole vs. closest_length_min_mae_value')
fig.add_trace(go.Scatter(x=[0, 50], y=[0, 50], mode='lines', name='line'))

#save the plot to png
fig.write_image(f'carapace pinhole vs. closest_length_min_mae_value.png')
#add a trendline

#add line


fig.show()
#
#

In [8]:
#min_mpe_focal_length from MPError_focal_min, MPError_focal_max, MPError_focal_median
# min_mae_focal_length from AbsError_focal_min, AbsError_focal_max, AbsError_focal_median

df['min_mpe_focal_length']=df[['MPError_focal_min', 'MPError_focal_max','MPError_focal_median']].min(axis=1)

df['max_mpe_focal_length']=df[['MPError_focal_min', 'MPError_focal_max','MPError_focal_median']].max(axis=1)

df['min_mae_focal_length']=df[['AbsError_focal_min', 'AbsError_focal_max','AbsError_focal_median']].min(axis=1)

df['max_mae_focal_length']=df[['AbsError_focal_min', 'AbsError_focal_max','AbsError_focal_median']].max(axis=1)


#print title of the prints
print('pond type count:')
print(df['Pond_Type'].value_counts())

print('pond type mean:')
print(df.groupby('Pond_Type')['Avg_Length'].mean())

print('pond type std:')
print(df.groupby('Pond_Type')['Avg_Length'].std())


print ('min_mpe fov median:')
print(df.groupby('Pond_Type')['min_mpe'].median())
#mean and std for each pond type of the lengt_1, length_2, length_

#print min_mae median and std for each pond type

print('min_mae fov median:')
print(df.groupby('Pond_Type')['min_mae'].median())



print('max_mpe fov median:')
print(df.groupby('Pond_Type')['max_mpe'].median())

print('max_mae fov median:')
print(df.groupby('Pond_Type')['max_mae'].median())


print('min_mpe focal length median:')
print(df.groupby('Pond_Type')['min_mpe_focal_length'].median())

print('max_mpe focal length median:')
print(df.groupby('Pond_Type')['max_mpe_focal_length'].median()) 

print('min_mae focal length median:')
print(df.groupby('Pond_Type')['min_mae_focal_length'].median())

print('max_mae focal length median:')   
print(df.groupby('Pond_Type')['max_mae_focal_length'].median())








pond type count:
Pond_Type
test-right    99
test-car      36
test-left     13
Name: count, dtype: int64
pond type mean:
Pond_Type
test-car      36.382602
test-left     24.400821
test-right    35.204135
Name: Avg_Length, dtype: float64
pond type std:
Pond_Type
test-car      5.056834
test-left     5.935468
test-right    5.529236
Name: Avg_Length, dtype: float64
min_mpe fov median:
Pond_Type
test-car      18.332959
test-left      8.182464
test-right     6.147389
Name: min_mpe, dtype: float64
min_mae fov median:
Pond_Type
test-car      6.451846
test-left     2.223462
test-right    2.121701
Name: min_mae, dtype: float64
max_mpe fov median:
Pond_Type
test-car      28.295583
test-left     20.843634
test-right    19.806458
Name: max_mpe, dtype: float64
max_mae fov median:
Pond_Type
test-car      11.455317
test-left      4.706797
test-right     6.940607
Name: max_mae, dtype: float64
min_mpe focal length median:
Pond_Type
test-car      17.297178
test-left      6.728062
test-right     5.131809
Na

In [18]:
#mean difference between min max of Length_1, Length_2, Length_3 for each pond type
print('mean difference between min max for each pond type:')
print(df.groupby('Pond_Type')['diff_max_min'].std())

mean difference between min max for each pond type:
Pond_Type
circular big male      3.423313
circular small male    3.619078
square-femal           3.387604
Name: diff_max_min, dtype: float64


In [16]:
#print uncertainty of the mean 
print('uncertainty of the mean:')
print(df.groupby('Pond_Type')['Std_Length'].mean()/np.sqrt(df.groupby('Pond_Type')['Avg_Length'].count()))

uncertainty of the mean:
Pond_Type
circular big male      0.261743
circular small male    0.649817
square-femal           0.428996
dtype: float64


In [ ]:
X = df[['camera_distance', 'pose_metric_1', 'pose_metric_2']]  # Add other pose metrics if available
X = sm.add_constant(X)
y = df['diff']
model = sm.OLS(y, X).fit()
print(model.summary())




melted_df = df.melt(id_vars=['Label','PrawnID'], value_vars=['Length_1', 'Length_2', 'Length_3'],
                   var_name='rater', value_name='measurement')
icc = pg.intraclass_corr(data=melted_df, targets=['Label','PrawnID'], raters='rater', ratings='measurement')
print(icc)


# Correlation Analysis
pearson_corr, pearson_p = pearsonr(df['keypoint_mm'], df['expert_mean'])
spearman_corr, spearman_p = spearmanr(df['keypoint_mm'], df['expert_mean'])
print(f"Pearson Correlation: {pearson_corr:.3f}, p-value: {pearson_p:.3e}")
print(f"Spearman Correlation: {spearman_corr:.3f}, p-value: {spearman_p:.3e}")

mean_measurements = (df['keypoint_mm'] + df['expert_mean']) / 2
diff_measurements = df['diff']
md = np.mean(diff_measurements)
sd = np.std(diff_measurements)

plt.figure(figsize=(10, 6))
plt.scatter(mean_measurements, diff_measurements, alpha=0.5)
plt.axhline(md, color='gray', linestyle='--', label=f'Mean Diff = {md:.2f} mm')
plt.axhline(md + 1.96*sd, color='red', linestyle='--', label=f'+1.96 SD = {md + 1.96*sd:.2f} mm')
plt.axhline(md - 1.96*sd, color='red', linestyle='--', label=f'-1.96 SD = {md - 1.96*sd:.2f} mm')
plt.xlabel('Mean of Measurements (mm)')
plt.ylabel('Difference (Keypoint - Expert) (mm)')
plt.title('Bland-Altman Plot')
plt.legend()
plt.show()



stat, p = shapiro(df['diff'])
print(f'Shapiro-Wilk Test: stat={stat:.3f}, p-value={p:.3e}')

# Homogeneity of Variance
# Assuming 'camera_distance_category' is already defined as in the visualization section
groups = df.groupby('camera_distance_category')['diff'].apply(list)
stat, p = levene(*groups)
print(f"Levene’s Test: stat={stat:.3f}, p-value={p:.3e}")
